In [34]:
import pandas as pd
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

In [35]:
# Veri setinizi yükleyin
df = pd.read_csv("e-ticaret_urun_yorumlari.csv", delimiter=';')
# Veri setini eğitim ve test setlerine bölün
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


In [36]:
# Türkçe BERT modelini ve belirtilen tokenizer'ı yükleyin
model_name = "dbmdz/bert-base-turkish-cased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = TFBertForSequenceClassification.from_pretrained(model_name)


tf_model.h5: 100%|██████████| 545M/545M [08:21<00:00, 1.09MB/s] 
All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [38]:
# Veri setini BERT için tokenize edin
def tokenize_data(data, tokenizer, max_length=128):
    return tokenizer(data, truncation=True, padding='max_length', max_length=max_length, return_tensors='tf')


In [39]:
# Eğitim ve test verilerini tokenize edin
train_data = tokenize_data(train_df['Metin'].tolist(), tokenizer)
test_data = tokenize_data(test_df['Metin'].tolist(), tokenizer)


In [40]:
# Etiketleri hazırlayın
train_labels = tf.convert_to_tensor(train_df['Durum'].tolist())
test_labels = tf.convert_to_tensor(test_df['Durum'].tolist())

In [41]:
# Modeli derleyin
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


In [42]:
# Modeli eğitin
model.fit(train_data, train_labels, epochs=3, batch_size=8)

Epoch 1/3


ValueError: Cannot generate a hashable key for IteratorSpec(({'input_ids': TensorSpec(shape=(8, 128), dtype=tf.int32, name=None), 'token_type_ids': TensorSpec(shape=(8, 128), dtype=tf.int32, name=None), 'attention_mask': TensorSpec(shape=(8, 128), dtype=tf.int32, name=None)}, TensorSpec(shape=(8,), dtype=tf.int32, name=None)),) because the _serialize() method returned an unsupproted value of type <class 'transformers.tokenization_utils_base.BatchEncoding'>

In [ ]:
 Modeli değerlendirin
eval_results = model.evaluate(test_data, test_labels)
print("Test Verileri İçin Doğruluk:", eval_results[1])



In [ ]:
# Test verileri üzerinde tahmin yapın
predictions = model.predict(test_data)
predicted_labels = predictions.logits.numpy().argmax(axis=1)

# Sınıflandırma raporunu görüntüleyin
print("Sınıflandırma Raporu:")
print(classification_report(test_labels.numpy(), predicted_labels))